In [2]:
import os
import sys

# adding classes folder to system path
sys.path.insert(0, os.path.abspath('..') + '/gispy')

import time
import statistics
import pandas as pd

from os import listdir
from os.path import isfile, join

from utils import get_wayback_url_content, get_wayback_urls

In [3]:
# adding classes folder to system path
sys.path.insert(0, os.path.abspath('..') + '/gispy')

df = pd.read_csv('../data/gist_annotation.csv')

df = df.loc[df['Answer.gist'].isin(['yes', 'no'])]

docs_path = '../data/gist'
txt_files = [f for f in listdir(docs_path) if isfile(join(docs_path, f)) and '.txt' in f]

In [4]:
yes = 0
no = 0
for txt_file in txt_files:
    if txt_file.endswith('_yes.txt'):
        yes += 1
    else:
        no += 1
print('yes: {}, no: {}'.format(yes, no))

yes: 147, no: 38


In [3]:
df_now = pd.read_excel('../data/gist.xlsx')
a = df.loc[~df['Input.url'].isin(list(df_now['Input.url']))]
sub_a = a[['Input.url', 'Answer.gist']]
sub_a.to_excel('../data/gist_extra.xlsx')

In [3]:
wayback_urls = get_wayback_urls(list(df['Input.url']))

0
20
40
60
80
100
120
140
160
180
200


In [4]:
wayback_urls_first = []
wayback_urls_last = []

for idx, row in df.iterrows():
    if row['Input.url'] in wayback_urls:
        value = wayback_urls[row['Input.url']]
        if len(value) == 0:
            wayback_urls_first.append('')
            wayback_urls_last.append('')
        if len(value) == 1:
            wayback_urls_first.append(value[0])
            wayback_urls_last.append(value[0])
        elif len(value) > 1:
            wayback_urls_first.append(value[0])
            wayback_urls_last.append(value[-1])
    else:
        wayback_urls_first.append('')
        wayback_urls_last.append('')

In [5]:
len(wayback_urls), len(wayback_urls_first), len(wayback_urls_last)

(214, 214, 214)

In [7]:
# wayback_urls, all_content = get_wayback_url_content(list(df['Input.url']), sleep_time=1)
# print('{}, {}, {}'.format(len(df), len(wayback_urls), len(all_content)))

In [6]:
# adding new cols to dataframe
df['wayback_url_first'] = wayback_urls_first
df['wayback_url_last'] = wayback_urls_last

# creating a subset of needed columns and filtering out rows with empty text
sub_df = df[['Input.url', 'wayback_url_first', 'wayback_url_last', 'Answer.gist']]
sub_df = sub_df.loc[~sub_df['wayback_url_first'].isin([''])]
sub_df = sub_df.reset_index(drop=True)

sub_df.to_excel('../data/gist.xlsx')

### Creating new empty text files for each URL

In [6]:
doc_folder = '../data/gist'
for idx, row in sub_df.iterrows():
    with open('{}/{}_{}.txt'.format(doc_folder, idx, row['Answer.gist']), 'w') as text_file:
        text_file.write('')

### Analyzing the results

In [2]:
df = pd.read_csv('../gispy/res.csv')
score_type = 'gis'

yes_scores = []
no_scores = []

for idx, row in df.iterrows():
    if '_yes.txt' in row['d_id']:
        yes_scores.append(row[score_type])
    else:
        no_scores.append(row[score_type])

print('avg \'gist=yes\' score: {}, stdev: {}'.format(sum(yes_scores) / len(yes_scores), statistics.stdev(yes_scores)))
print('avg \'gist=no\' score: {}, stdev: {}'.format(sum(no_scores) / len(no_scores), statistics.stdev(no_scores)))

avg 'gist=yes' score: -0.7386003404867852, stdev: 0.6099601563692668
avg 'gist=no' score: -0.7155898134147554, stdev: 0.5371883697342696


In [3]:
len(yes_scores), len(no_scores)

(46, 15)

### [Temporary] converting old indices to the new list of indices

In [8]:
df_old = pd.read_excel('../data/gist_v1.xlsx')
df_new = pd.read_excel('../data/gist.xlsx')

for idx, row in df_new.iterrows():
    a = df_old.loc[df_old['Input.url']==row['Input.url']]
    if len(a) == 1:
        # we found the very record
        label = a.iloc[0]['Answer.gist']
        with open('../data/text/{}_{}.txt'.format(a.index[0], label), 'r') as in_file:
            text = in_file.read()
            
        b = df_new.loc[df_new['Input.url']==row['Input.url']]
        if len(b) == 1:
            label_2 = b.iloc[0]['Answer.gist']
            if label == label_2:
                with open('../data/gist/{}_{}.txt'.format(b.index[0], label_2), 'w') as out_file:
                    out_file.write(text)

### Double-checking labels

In [2]:
df = pd.read_excel('../data/gist.xlsx')
docs_path = '../data/gist'
txt_files = [f for f in listdir(docs_path) if isfile(join(docs_path, f)) and '.txt' in f]

id_label = {}
for txt_file in txt_files:
    file_id = txt_file.split('_')[0]
    file_label = txt_file.split('_')[1].strip('.txt')
    id_label[file_id] = file_label

# if there's an incompatibility between file id and label, "false" will be printed
for idx, row in df.iterrows():
    if idx in id_label and id_label[idx] != row['Answer.gist']:
        print('false')